<a href="https://colab.research.google.com/github/omkar-salunke/People_counting_basic/blob/main/SOE_TALLY_Daybook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# prompt: list of files in the path in python

import os

path = '/content/drive/My Drive/2024_25'  # Replace with your desired path
files = os.listdir(path)
print(files)


['MEBook.xml', 'SOE.XML', 'MEDayBook.xlsx', 'SOE_daybook.xlsx', 'SOEDayBook_2024-10-12.xlsx']


In [4]:
xml_file = '/content/drive/My Drive/2024_25/SOE.XML'


In [5]:
import xml.etree.ElementTree as ET
import pandas as pd

# Function to recursively extract text and sub-elements from a node
def extract_element_data(element):
    element_data = {}
    # If the element has no children, return its text
    if len(element) == 0:
        return element.text

    # Otherwise, go deeper into the element's children
    for child in element:
        # Recursively extract data for each child element
        child_data = extract_element_data(child)
        element_data[child.tag] = child_data
    return element_data

# Function to flatten nested dictionaries (e.g., 'amount': {'P': {'T': 100}})
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Function to recursively search for all 'voucher' elements and extract their data
def find_all_vouchers(element, voucher_tag="VOUCHER"):
    vouchers = []

    # If the current element is a 'voucher', collect its data
    if element.tag == voucher_tag:
        voucher_dict = {}
        # Collect data for all child elements within the voucher
        for sub_child in element:
            extracted_data = extract_element_data(sub_child)
            voucher_dict[sub_child.tag] = extracted_data

        # Flatten the voucher dictionary to handle nested elements
        voucher_dict_flattened = flatten_dict(voucher_dict)
        vouchers.append(voucher_dict_flattened)

    # Recursively search in all children of the current element
    for child in element:
        vouchers.extend(find_all_vouchers(child, voucher_tag))

    return vouchers

# Function to convert the XML to a DataFrame by extracting all vouchers
def xml_to_dataframe_vouchers(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Find all 'voucher' elements in the XML
    voucher_list = find_all_vouchers(root)

    # Create a DataFrame from the list of voucher dictionaries
    df = pd.DataFrame(voucher_list)
    return df

# Path to your XML file
# xml_file = 'path_to_your_tally_daybook.xml'

# Convert XML to DataFrame
df = xml_to_dataframe_vouchers(xml_file)

# Display the DataFrame
print(df.head())


                         ADDRESS.LIST.ADDRESS  \
0   S-70 M.I.D.C. ,HINGNA ROAD, NAGPUR-440016   
1   S-70 M.I.D.C. ,HINGNA ROAD, NAGPUR-440016   
2   S-70 M.I.D.C. ,HINGNA ROAD, NAGPUR-440016   
3  M-22/1,23M.I.D.C.HINGNA INDUSTRIAL ,NAGPUR   
4  M-22/1,23M.I.D.C.HINGNA INDUSTRIAL ,NAGPUR   

  DISPATCHFROMADDRESS.LIST.DISPATCHFROMADDRESS  \
0                           Maharashtra-431136   
1                           Maharashtra-431136   
2                           Maharashtra-431136   
3                           Maharashtra-431136   
4                           Maharashtra-431136   

     BASICBUYERADDRESS.LIST.BASICBUYERADDRESS  \
0   S-70 M.I.D.C. ,HINGNA ROAD, NAGPUR-440016   
1   S-70 M.I.D.C. ,HINGNA ROAD, NAGPUR-440016   
2   S-70 M.I.D.C. ,HINGNA ROAD, NAGPUR-440016   
3  M-22/1,23M.I.D.C.HINGNA INDUSTRIAL ,NAGPUR   
4  M-22/1,23M.I.D.C.HINGNA INDUSTRIAL ,NAGPUR   

  OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS      DATE IRNACKDATE  \
0                                     -1  20

In [6]:
df.columns.tolist()

['ADDRESS.LIST.ADDRESS',
 'DISPATCHFROMADDRESS.LIST.DISPATCHFROMADDRESS',
 'BASICBUYERADDRESS.LIST.BASICBUYERADDRESS',
 'OLDAUDITENTRYIDS.LIST.OLDAUDITENTRYIDS',
 'DATE',
 'IRNACKDATE',
 'GUID',
 'GSTREGISTRATIONTYPE',
 'VATDEALERTYPE',
 'STATENAME',
 'VOUCHERTYPENAME',
 'COUNTRYOFRESIDENCE',
 'PARTYGSTIN',
 'PLACEOFSUPPLY',
 'CLASSNAME',
 'PARTYNAME',
 'PARTYLEDGERNAME',
 'PARTYMAILINGNAME',
 'PARTYPINCODE',
 'IRN',
 'BILLTOPLACE',
 'DISPATCHFROMNAME',
 'DISPATCHFROMSTATENAME',
 'DISPATCHFROMPINCODE',
 'DISPATCHFROMPLACE',
 'SHIPTOPLACE',
 'CONSIGNEEGSTIN',
 'CONSIGNEEMAILINGNAME',
 'CONSIGNEEPINCODE',
 'CONSIGNEESTATENAME',
 'VOUCHERNUMBER',
 'BASICBASEPARTYNAME',
 'CSTFORMISSUETYPE',
 'CSTFORMRECVTYPE',
 'FBTPAYMENTTYPE',
 'PERSISTEDVIEW',
 'IRNACKNO',
 'IRNQRCODE',
 'BASICBUYERNAME',
 'BASICSHIPVESSELNO',
 'CONSIGNEECOUNTRYNAME',
 'VCHGSTCLASS',
 'VCHENTRYMODE',
 'DIFFACTUALQTY',
 'ISMSTFROMSYNC',
 'ISDELETED',
 'ISSECURITYONWHENENTERED',
 'ASORIGINAL',
 'AUDITED',
 'FORJOBCOSTING'

In [8]:
import os
from datetime import datetime
old_file_path  = '/content/drive/My Drive/2024_25/SOEDayBook.xlsx'
today_date = datetime.today().strftime('%Y-%m-%d')
if 'SOEDayBook.xlsx'  in files:
    # Create the new file name by appending today's date
  file_name, file_extension = os.path.splitext(old_file_path)
  new_file_path = f"{file_name}_{today_date}{file_extension}"

  # Rename the file
  os.rename(old_file_path, new_file_path)
  if not os.path.exists(old_file_path) and os.path.exists(new_file_path):
      print(f"File renamed and saved as: {new_file_path}")
      file_sucess = True

  else:
      print(f"Error renaming file: {old_file_path}")
else:
  file_sucess = True

In [9]:
import pandas as pd
if file_sucess == True:

  df.to_excel("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",sheet_name='master_sheet')

  for xrow in df['VOUCHERTYPENAME'].unique().tolist():
    with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
      df[df['VOUCHERTYPENAME']==xrow].reset_index(drop=True).to_excel(writer,sheet_name=xrow)
  df['DATE'] = pd.to_datetime(df['DATE'])

  df_sales = df[df['VOUCHERTYPENAME']=='SALES GST'].reset_index(drop=True)
  # Calculate new dates
  df_sales['DATE_plus_45'] = df_sales['DATE'] + pd.DateOffset(days=45)
  df_sales['DATE_plus_60'] = df_sales['DATE'] + pd.DateOffset(days=60)
  df_sales['DATE_plus_90'] = df_sales['DATE'] + pd.DateOffset(days=90)
  df_sales['Week_Number'] = df_sales['DATE_plus_45'].dt.isocalendar().week
  df_sales['ALLINVENTORYENTRIES.LIST.AMOUNT'] = df_sales['ALLINVENTORYENTRIES.LIST.AMOUNT'].astype(float)
  df_sales = df_sales.rename(columns={'ALLINVENTORYENTRIES.LIST.AMOUNT': 'sales_amount'})
  df_sales_g = df_sales.groupby(['PARTYLEDGERNAME','Week_Number'])['sales_amount'].sum().reset_index()

  # Add a column for the payment due date (45 days after the sale date)
  df_sales['due_date'] = df_sales['DATE'] + pd.Timedelta(days=45)

  # Filter the DataFrame to include only pending payments (due date is in the future)
  pending_payments = df_sales[df_sales['due_date'] > pd.Timestamp.now()]

  # Group by customer name and week of the year, and sum the amounts
  pending_payments_per_week = pending_payments.groupby(['PARTYLEDGERNAME'])['sales_amount'].sum().reset_index()

  pending_payments_per_week.columns = ['PARTYLEDGERNAME', 'due_amount_'+str(pd.Timestamp.now().date())]
  outstanding_payments_g = df_sales.groupby(['PARTYLEDGERNAME'])['sales_amount'].sum().reset_index()

  outstanding_payments_g.columns = ['PARTYLEDGERNAME', 'outstanding_amount']
  df_receipt = df[df['VOUCHERTYPENAME']=='Receipt'].reset_index(drop=True)
  df_receipt['ALLLEDGERENTRIES.LIST.AMOUNT'] = df_receipt['ALLLEDGERENTRIES.LIST.AMOUNT'].astype(float)
  df_receipt = df_receipt.rename(columns={'ALLLEDGERENTRIES.LIST.AMOUNT': 'receipt_amount'})
  df_receipt_g = df_receipt.groupby(['PARTYLEDGERNAME'])['receipt_amount'].sum().reset_index()
  df_due_45_days = pd.merge(df_receipt_g,outstanding_payments_g,on=['PARTYLEDGERNAME'],how='right')
  df_due_45_days = pd.merge(df_due_45_days,pending_payments_per_week,on=['PARTYLEDGERNAME'],how='left')
  df_due_45_days['closing_balance_on_31_march'] = ''
  with pd.ExcelWriter("/content/drive/My Drive/2024_25/SOE_daybook.xlsx",mode='a') as writer:
    df_due_45_days.to_excel(writer,sheet_name='Due_45_days')
